# Analyzing Crunchbase Data in chunks using sqlite and pandas
## Introduction
Constraint : use maximum of 10MB of memory

data can be found [here](https://github.com/datahoarder/crunchbase-october-2013/blob/master/crunchbase-investments.csv)
## Chunk Size testing

In [38]:
import pandas as pd
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
total=0
for chunk in chunk_iter:
    mem = chunk.memory_usage(deep=True).sum()/1024**2
    total+=mem
    print(mem)
print('Total:', total)

5.579195022583008
5.528186798095703
5.535004615783691
5.528155326843262
5.524299621582031
5.553397178649902
5.531391143798828
5.509613037109375
5.396082878112793
4.63945198059082
2.663668632507324
Total: 56.98844623565674


We are good at 5000 size chunks, since our memory usage is around 5MB per chunk

## Counting missing values in chunks

In [27]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
missing = []
total = 0
for chunk in chunk_iter:
    missing.append(chunk.isnull().sum())
    total+=len(chunk)
conc = pd.concat(missing)
print(conc.groupby(conc.index).sum())
print('total rows:', total)

company_category_code       643
company_city                533
company_country_code          1
company_name                  1
company_permalink             1
company_region                1
company_state_code          492
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
funding_round_type            3
investor_category_code    50427
investor_city             12480
investor_country_code     12001
investor_name                 2
investor_permalink            2
investor_region               2
investor_state_code       16809
raised_amount_usd          3599
dtype: int64
total rows: 52870


## Drop useless columns
Dont need URLS, or large numbers of missing values

In [39]:
to_drop = ['investor_permalink', 'company_permalink', 'investor_category_code']
to_use = chunk.columns.drop(to_drop).to_list()
to_use

['company_name',
 'company_category_code',
 'company_country_code',
 'company_state_code',
 'company_region',
 'company_city',
 'investor_name',
 'investor_country_code',
 'investor_state_code',
 'investor_region',
 'investor_city',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year',
 'raised_amount_usd']

## Optimize datatypes
Let's take another look at the data

In [33]:
small_data = pd.read_csv('crunchbase-investments.csv', usecols=to_use, nrows=5)
print(small_data.info())
small_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 17 columns):
company_name             5 non-null object
company_category_code    5 non-null object
company_country_code     5 non-null object
company_state_code       4 non-null object
company_region           5 non-null object
company_city             4 non-null object
investor_name            5 non-null object
investor_country_code    5 non-null object
investor_state_code      5 non-null object
investor_region          5 non-null object
investor_city            5 non-null object
funding_round_type       5 non-null object
funded_at                5 non-null object
funded_month             5 non-null object
funded_quarter           5 non-null object
funded_year              5 non-null int64
raised_amount_usd        5 non-null int64
dtypes: int64(2), object(15)
memory usage: 760.0+ bytes
None


,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,AdverCar,advertising,USA,CA,SF Bay,San Francisco,1-800-FLOWERS.COM,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,LaunchGram,news,USA,CA,SF Bay,Mountain View,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,uTaP,messaging,USA,NaN,United States - Other,NaN,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,ZoopShop,software,USA,OH,Columbus,columbus,10Xelerator,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,eFuneral,web,USA,OH,Cleveland,Cleveland,10Xelerator,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000


## Optimizing numeric types
Let's find out if funded year and raised amount can be changed to more efficient datatypes. Since they have missing values, we make sure they are float accorss all chunks then downcast.

In [51]:
import numpy as np
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=to_use)
total=0
for chunk in chunk_iter:
    chunk['funded_year'] = chunk['funded_year'].astype('float')
    chunk['funded_year'] = pd.to_numeric(chunk['funded_year'], downcast='float')
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].astype('float')
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'], downcast='float')
    mem = chunk.memory_usage(deep=True).sum()/1024**2
    total+=mem

print('Total:', total)
chunk.dtypes
            

Total: 47.37186145782471


company_name              object
company_category_code     object
company_country_code      object
company_state_code        object
company_region            object
company_city              object
investor_name             object
investor_country_code    float64
investor_state_code      float64
investor_region           object
investor_city            float64
funding_round_type        object
funded_at                 object
funded_month              object
funded_quarter            object
funded_year              float32
raised_amount_usd        float32
dtype: object

## Optimizing object types
From looking at the data above, we can already tell:
* funded at and funded month can be read as datetypes to save memoery
* funded quarter contains redundant info and can be converted to just a float type with the last number (sadly not int due to missing values)
* there might be others than can be converted to category depending on their value counts -- lets identify those. A threshold of 51 is good to keep all the USA states as categories, but avoid illogical categories

In [56]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=to_use)

col_values= {}

for chunk in chunk_iter:
    objects_only = chunk.select_dtypes(include='object')
    for col in objects_only.columns:
        if col in col_values:
            col_values[col].append(objects_only[col].value_counts())
        else:
            col_values[col] = [objects_only[col].value_counts()]
            
to_category = []
for col in col_values:
    conc = pd.concat(col_values[col])
    uniques = conc.groupby(conc.index).sum()
    percentage = len(uniques) / conc.sum() *100
    if len(uniques) <= 50:
        print(col, len(uniques), conc.sum(), round(percentage,2))
        to_category.append(col)
        
col_dtypes = {}
for col in to_category:
    col_dtypes[col] = 'category'
    
print('\n', col_dtypes)

company_category_code 43 52227 0.08
company_country_code 2 52869 0.0
company_state_code 50 52378 0.1
investor_state_code 50 36061 0.14
funding_round_type 9 52867 0.02

 {'company_category_code': 'category', 'company_country_code': 'category', 'company_state_code': 'category', 'investor_state_code': 'category', 'funding_round_type': 'category'}


### chunk processing with numeric and object optimizations

In [59]:
to_date = ['funded_at', 'funded_month']
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=to_use, dtype = col_dtypes, parse_dates=to_date)

total=0
for chunk in chunk_iter:
    chunk['funded_year'] = chunk['funded_year'].astype('float')
    chunk['funded_year'] = pd.to_numeric(chunk['funded_year'], downcast='float')
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].astype('float')
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'], downcast='float')
    
    chunk['funded_quarter'] = chunk['funded_quarter'].str[-1].astype('float')
    chunk['funded_quarter'] = pd.to_numeric(chunk['funded_quarter'], downcast='float')
    mem = chunk.memory_usage(deep=True).sum()/1024**2
    total+=mem

print('Total:', total)
chunk.head()

Total: 24.057799339294434


,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10-01,4.0,2012.0,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10-01,4.0,2007.0,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04-01,2.0,2008.0,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01-01,1.0,2010.0,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01-01,1.0,2010.0,NaN


We have reduced total memory down to 24MB
## Loading into SQlite

In [60]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=to_use, dtype = col_dtypes, parse_dates=to_date)

for chunk in chunk_iter:
    chunk['funded_year'] = chunk['funded_year'].astype('float')
    chunk['funded_year'] = pd.to_numeric(chunk['funded_year'], downcast='float')
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].astype('float')
    chunk['raised_amount_usd'] = pd.to_numeric(chunk['raised_amount_usd'], downcast='float')
    
    chunk['funded_quarter'] = chunk['funded_quarter'].str[-1].astype('float')
    chunk['funded_quarter'] = pd.to_numeric(chunk['funded_quarter'], downcast='float')

    chunk.to_sql("investments", conn, if_exists='append', index=False)

In [66]:
pd.read_sql('PRAGMA table_info(investments)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,company_name,TEXT,0,None,0
1,1,company_category_code,TEXT,0,None,0
2,2,company_country_code,TEXT,0,None,0
3,3,company_state_code,TEXT,0,None,0
4,4,company_region,TEXT,0,None,0
5,5,company_city,TEXT,0,None,0
6,6,investor_name,TEXT,0,None,0
7,7,investor_country_code,TEXT,0,None,0
8,8,investor_state_code,TEXT,0,None,0
9,9,investor_region,TEXT,0,None,0


### Checking size of db file

In [72]:
import os
statinfo = os.stat('crunchbase.db')
statinfo.st_size / 1024**2

7.8515625

## Data analysis in pandas, pulling the data from the db file in chunks
Questions to answer:
* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

### First Questions
What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.

In [93]:
query = 'SELECT company_name, SUM(raised_amount_usd) raised FROM investments GROUP BY company_name'
chunk_iter = pd.read_sql(query, conn, chunksize=5000)
raised_amount = []
for chunk in chunk_iter:
    raised_amount.append(chunk)
conc = pd.concat(raised_amount)
total_funds_raised_by_company = conc.groupby('company_name').sum().sort_values(by='raised', ascending=False)

threshold = int(.1*len(total_funds_raised_by_company))
top_10_percent = total_funds_raised_by_company.iloc[:threshold,:]
top_10_percent_share = top_10_percent['raised'].sum() / total_funds_raised_by_company['raised'].sum()
top_10_percent_share

0.6712772709939844

In [94]:
threshold = int(.01*len(total_funds_raised_by_company))
top_1_percent = total_funds_raised_by_company.iloc[:threshold,:]
top_1_percent_share = top_1_percent['raised'].sum() / total_funds_raised_by_company['raised'].sum()
top_1_percent_share

0.26217737281266595

In [98]:
threshold = int(.9*len(total_funds_raised_by_company))
bottom_10_percent = total_funds_raised_by_company.iloc[threshold:,:]
bottom_10_percent_share = bottom_10_percent['raised'].sum() / total_funds_raised_by_company['raised'].sum()
bottom_10_percent_share

0.0

In [99]:
threshold = int(.99*len(total_funds_raised_by_company))
bottom_1_percent = total_funds_raised_by_company.iloc[threshold:,:]
bottom_1_percent_share = bottom_1_percent['raised'].sum() / total_funds_raised_by_company['raised'].sum()
bottom_1_percent_share

0.0

### Second Question
Which category of company attracted the most investments?

In [102]:
query = 'SELECT company_category_code, SUM(raised_amount_usd) raised FROM investments GROUP BY company_category_code'
chunk_iter = pd.read_sql(query, conn, chunksize=5000)
raised_amount = []
for chunk in chunk_iter:
    raised_amount.append(chunk)
conc = pd.concat(raised_amount)
total_funds_raised_by_category = conc.groupby('company_category_code').sum().sort_values(by='raised', ascending=False)
total_funds_raised_by_category.iloc[0]

raised    1.103964e+11
Name: biotech, dtype: float64

### Third Question
Which investor contributed the most money (across all startups)?

In [106]:
query = 'SELECT investor_name, SUM(raised_amount_usd) raised FROM investments GROUP BY investor_name'
chunk_iter = pd.read_sql(query, conn, chunksize=5000)
raised_amount = []
for chunk in chunk_iter:
    raised_amount.append(chunk)
conc = pd.concat(raised_amount)
total_funds_raised_by_investor = conc.groupby('investor_name').sum().sort_values(by='raised', ascending=False)
total_funds_raised_by_investor.iloc[0]

raised    1.121783e+10
Name: Kleiner Perkins Caufield & Byers, dtype: float64

### Fourth Question
Which investors contributed the most money per startup?

In [150]:
query = 'SELECT company_name, investor_name, raised_amount_usd raised FROM investments'
chunk_iter = pd.read_sql(query, conn, chunksize=5000)
raised_amount = []
for chunk in chunk_iter:
    raised_amount.append(chunk)
conc = pd.concat(raised_amount)
total_funds_by_comp_by_investor = conc.groupby(['company_name', 'investor_name']).sum()

In [163]:
total_funds_by_comp_by_investor = total_funds_by_comp_by_investor.reset_index()
best_investor_by_comp = total_funds_by_comp_by_investor.iloc[total_funds_by_comp_by_investor.groupby(['company_name'])['raised'].idxmax()]
best_investor_by_comp.drop('index', axis=1)

,level_0,company_name,investor_name,raised
0,0,#waywire,First Round Capital,1750000.0
5,5,0xdata,Nexus Venture Partners,1700000.0
6,6,1-800-DENTIST,Bain Capital Ventures,0.0
8,8,1000memories,Caterina Fake,2520000.0
18,18,100Plus,Band of Angels,750000.0
22,22,1010data,Norwest Venture Partners,35000000.0
23,23,11i Solutions,Steel Pier Capital Advisors,1800000.0
26,26,121nexus,"Mayfly Capital, LLC",500000.0
29,29,12Society,Daniel Broukhim,0.0
35,35,1366 Technologies,North Bridge Venture Partners,43550000.0


### Fifth Question
Which funding round was the most popular? Which was the least popular?

In [167]:
query = 'SELECT funding_round_type, SUM(raised_amount_usd) raised, COUNT(*) total_rows FROM investments GROUP BY funding_round_type'
chunk_iter = pd.read_sql(query, conn, chunksize=5000)
raised_amount = []
for chunk in chunk_iter:
    raised_amount.append(chunk)
conc = pd.concat(raised_amount)
total_funds_raised_by_round = conc.groupby('funding_round_type').sum().sort_values(by='total_rows', ascending=False)
total_funds_raised_by_round

,raised,total_rows
funding_round_type,,
series-a,8.654215e+10,13938
series-c+,2.657535e+11,10870
angel,4.962075e+09,8989
venture,1.305565e+11,8917
series-b,1.283268e+11,8794
other,1.850726e+10,964
private-equity,1.615988e+10,357
post-ipo,3.091760e+10,33
crowdfunding,6.491500e+06,5
